<h1>Data preprocessing</h1>

<h2>Importing necessary modules</h2>

In [20]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold


from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=True
from rdkit.Chem import Descriptors
import pickle as pk


In [3]:
# Define filepath to untested_molecules.csv.
path = r"untested_molecules.csv"
# Open the csv file and store the data as a list
csv_file = open(path, 'r')
reader = csv.reader(csv_file)
data = [row for row in reader]


<h2>Create DataFrame containing molecules and descriptors</h2>
<p>Possible to export it as csv file.</p>

In [4]:
# TODO: more sensible variable names?
m = Chem.MolFromSmiles(data[1][0])
vs = Descriptors.CalcMolDescriptors(m)

df_full = pd.DataFrame(vs.items())
df_full.rename(columns={1:data[1][0]},inplace=True)

for i,x in enumerate(data[2:]):
    m = Chem.MolFromSmiles(x[0])
    vs = Descriptors.CalcMolDescriptors(m)
    df_full[x[0]] = vs.values()

df_full.to_csv('descriptors_untested_molecules.csv')

C:\Users\20203148\AppData\Local\Temp\ipykernel_4456\2098505618.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[x[0]] = vs.values()
C:\Users\20203148\AppData\Local\Temp\ipykernel_4456\2098505618.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full[x[0]] = vs.values()
C:\Users\20203148\AppData\Local\Temp\ipykernel_4456\2098505618.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 

<p>DataFrame has to be read in from previous save, as rest of the program bugs out otherwise :(</p>

In [8]:
df_full = pd.read_csv('descriptors_untested_molecules.csv')
df_full.head()

,Unnamed: 0,0,CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,CC1(C)CC(=O)C2=C(C1)OC([NH3+])=C(C#N)[C@@]21C(=O)N(CC(N)=O)c2ccccc21,O=C(NCc1cccs1)C1CCC(CNS(=O)(=O)c2cccs2)CC1,O=C(c1ccc(S(=O)(=O)N2CCCCCC2)cc1)N1CCN(c2ccccn2)CC1,...,CC(C)CNC(=O)COC(=O)/C=C/c1ccccc1[N+](=O)[O-],CCOC(=O)N1CCN(C(=O)c2cccc(OC)c2OC)CC1,COc1cccc(CNC2=[NH+]c3ccccc3CS2)c1,COc1ccc(N2CC[NH+](CC(=O)Nc3c(C)cc(C)cc3C)CC2)cc1,CC(C)NC(=S)N/N=C1\CCCCC1C1CCCCC1,CCNC(=O)c1nc[nH]c1C(=O)NCC,CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,CC(C)C(C(=O)NCc1ccco1)N(C(=O)CNS(=O)(=O)c1ccccc1)c1ccccc1
0,0,MaxAbsEStateIndex,12.089288,12.797328,12.973159,11.709157,12.580327,13.814569,12.255903,12.912813,...,11.481222,12.705510,5.237728,12.532657,5.264541,11.472962,12.326979,12.799793,8.776692,13.305404
1,1,MaxEStateIndex,12.089288,12.797328,12.973159,11.709157,12.580327,13.814569,12.255903,12.912813,...,11.481222,12.705510,5.237728,12.532657,5.264541,11.472962,12.326979,12.799793,8.776692,13.305404
2,2,MinAbsEStateIndex,0.112255,0.238255,0.116476,0.004405,0.039856,0.033587,0.046403,0.060749,...,0.109976,0.134945,0.786409,0.095325,0.357127,0.127593,0.060040,0.140326,0.212513,0.056168
3,3,MinEStateIndex,-0.251126,-3.572690,-0.893827,-0.628881,-4.469682,-1.667616,-3.389516,-3.495798,...,-0.738104,-0.334667,0.786409,0.095325,0.357127,-0.347219,-0.947497,-3.620036,0.212513,-3.887541
4,4,qed,0.349176,0.664725,0.499889,0.247581,0.376568,0.748309,0.751930,0.748473,...,0.358983,0.842412,0.902451,0.848207,0.610458,0.648660,0.323364,0.460041,0.665075,0.474607


<h2>Transpose df_combined and slightly alter the dataset in a convenient format</h2>

In [22]:
#First we transpose df_combined to make the next steps more convienient 
df_transposed = df_full.transpose()

#Use the second row as column indices and then drop the first two rows afterwards
df_transposed.columns = df_transposed.iloc[1]
df_transposed = df_transposed.drop(df_transposed.index[0:2])
#rename the first column
df_transposed.columns.name = 'MolecularStructure'

#rename the dataframe to DF for further use
DF = df_transposed
DF

MolecularStructure,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,12.089288,12.089288,0.112255,-0.251126,0.349176,379.504,350.272,379.225977,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,12.797328,12.797328,0.238255,-3.57269,0.664725,442.537,416.329,442.156243,164.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,12.973159,12.973159,0.116476,-0.893827,0.499889,359.313,345.201,359.091749,134.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,11.709157,11.709157,0.004405,-0.628881,0.247581,358.374,338.214,358.139747,136.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,12.580327,12.580327,0.039856,-4.469682,0.376568,334.729,320.617,334.080823,120.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCNC(=O)c1nc[nH]c1C(=O)NCC,11.472962,11.472962,0.127593,-0.347219,0.64866,210.237,196.125,210.111676,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,12.326979,12.326979,0.06004,-0.947497,0.323364,464.519,432.263,464.227099,182.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,12.799793,12.799793,0.140326,-3.620036,0.460041,444.538,424.378,444.092597,156.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,8.776692,8.776692,0.212513,0.212513,0.665075,302.822,277.622,302.151748,114.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


<h2>pre-processing</h2>

First we remove all low variance features by using the same variance_mask as used on the tested molecules

In [24]:
%store -r variance_mask

DF_filtered = DF.loc[:, variance_mask]

DF_filtered

MolecularStructure,MaxAbsEStateIndex,MaxEStateIndex,MinEStateIndex,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_LOGPLOW,...,fr_ester,fr_ether,fr_halogen,fr_ketone,fr_methoxy,fr_para_hydroxylation,fr_pyridine,fr_quatN,fr_sulfide,fr_unbrch_alkane
CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,12.089288,12.089288,-0.251126,379.504,350.272,379.225977,148.0,2.142857,16.155548,-2.257675,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,12.797328,12.797328,-3.57269,442.537,416.329,442.156243,164.0,2.387097,32.233427,-2.409391,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,12.973159,12.973159,-0.893827,359.313,345.201,359.091749,134.0,2.538462,19.142146,-2.347503,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,11.709157,11.709157,-0.628881,358.374,338.214,358.139747,136.0,2.346154,16.678907,-2.7579,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,12.580327,12.580327,-4.469682,334.729,320.617,334.080823,120.0,2.545455,35.495692,-2.329215,...,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCNC(=O)c1nc[nH]c1C(=O)NCC,11.472962,11.472962,-0.347219,210.237,196.125,210.111676,82.0,2.466667,16.154385,-2.288957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,12.326979,12.326979,-0.947497,464.519,432.263,464.227099,182.0,2.151515,16.547882,-2.534544,...,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,12.799793,12.799793,-3.620036,444.538,424.378,444.092597,156.0,2.733333,32.233453,-2.401346,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,8.776692,8.776692,0.212513,302.822,277.622,302.151748,114.0,2.35,35.495692,-3.095879,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


We also remove the features that were found to be highly correlated in the testes molecules dataset

In [25]:
%store -r collinear_features
#make a list of features to drop
features_to_drop = list(set(pair[0] for pair in collinear_features))

#drop the columns corresponding to the features and save the new DF as DF_final
DF_final = DF_filtered.drop(columns = features_to_drop)
DF_final


MolecularStructure,MaxEStateIndex,MinEStateIndex,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,BertzCT,...,fr_ester,fr_ether,fr_halogen,fr_ketone,fr_methoxy,fr_para_hydroxylation,fr_pyridine,fr_quatN,fr_sulfide,fr_unbrch_alkane
CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,12.089288,-0.251126,2.142857,16.155548,-2.257675,6.006437,-0.126034,2.742286,1.791107,757.931608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,12.797328,-3.57269,2.387097,32.233427,-2.409391,7.887355,-0.152691,3.217778,1.286783,1073.400608,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,12.973159,-0.893827,2.538462,19.142146,-2.347503,6.236926,-0.383654,2.895389,1.89516,884.467127,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,11.709157,-0.628881,2.346154,16.678907,-2.7579,5.772305,-0.72472,2.575728,1.831575,810.970252,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,12.580327,-4.469682,2.545455,35.495692,-2.329215,6.325608,-0.137383,2.468395,2.088043,593.438266,...,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCNC(=O)c1nc[nH]c1C(=O)NCC,11.472962,-0.347219,2.466667,16.154385,-2.288957,6.041917,0.091593,2.66472,2.857416,326.249247,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,12.326979,-0.947497,2.151515,16.547882,-2.534544,5.919847,-0.14646,2.82264,2.331557,816.22218,...,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,12.799793,-3.620036,2.733333,32.233453,-2.401346,7.99672,-0.113362,2.782133,1.428075,1158.940027,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,8.776692,0.212513,2.35,35.495692,-3.095879,6.316629,-0.879944,2.398221,2.185,389.100254,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


<h2>Scale the data using a MinMax scaler to prepare for PCA</h2>

In [26]:
scaler = MinMaxScaler()
DF_final[:] = scaler.fit_transform(DF_final[:])
DF_final

MolecularStructure,MaxEStateIndex,MinEStateIndex,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,AvgIpc,BalabanJ,BertzCT,...,fr_ester,fr_ether,fr_halogen,fr_ketone,fr_methoxy,fr_para_hydroxylation,fr_pyridine,fr_quatN,fr_sulfide,fr_unbrch_alkane
CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,0.769783,0.792266,0.52381,0.016594,0.560809,0.169113,0.269036,0.582937,0.187068,0.411122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,0.82453,0.360167,0.621505,0.159348,0.475707,0.361923,0.26084,0.777046,0.062138,0.590601,...,0.25,0.166667,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0
COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,0.838126,0.708657,0.682051,0.043111,0.510422,0.19274,0.189832,0.645438,0.212843,0.483112,...,0.0,0.166667,0.125,0.0,0.2,0.0,0.0,0.0,0.0,0.0
Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,0.740391,0.743124,0.605128,0.02124,0.280217,0.145112,0.084973,0.514943,0.197093,0.441297,...,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,0.807751,0.243479,0.684848,0.188314,0.52068,0.201831,0.265546,0.471127,0.260624,0.317538,...,0.0,0.0,0.5,0.0,0.0,0.0,0.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCNC(=O)c1nc[nH]c1C(=O)NCC,0.722128,0.779765,0.653333,0.016583,0.543263,0.17275,0.335944,0.551272,0.451211,0.165527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,0.788162,0.701676,0.527273,0.020077,0.405505,0.160236,0.262756,0.615739,0.320947,0.444285,...,0.25,0.333333,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,0.824721,0.354008,0.76,0.159348,0.48022,0.373134,0.272931,0.599203,0.097139,0.639267,...,0.0,0.166667,0.0,0.0,0.0,0.25,0.0,0.0,0.333333,0.0
Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,0.513647,0.85258,0.606667,0.188314,0.090634,0.20091,0.037251,0.442479,0.284642,0.201285,...,0.0,0.333333,0.125,0.0,0.0,0.0,0.0,0.5,0.0,0.3


In [27]:
pca_reload = pk.load(open("pca.pkl",'rb'))
pca_results = pca_reload.transform(DF_final)

Finally we now create our finally iteration of the dataframe by selecting the desired amount of principal components and reintroducing the ALDH1_inhibition scores

In [29]:
#Create new reduced dataframe with PCA scores

df_reduced = pd.DataFrame(pca_results)

PC_column_names = [f'PC{i}'for i in range(1,34)]
df_reduced.columns = PC_column_names
df_reduced.index = DF.index

df_reduced

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
CCCCC/C(=N\NC(=O)CCC(=O)NCc1ccccc1)c1ccccc1,0.264705,-0.035380,-0.734212,0.042253,-0.334140,0.114293,-0.367323,0.034094,-0.270585,-0.475224,...,0.019112,0.137160,0.106412,0.069768,0.121549,0.045777,-0.110404,-0.218882,0.075640,0.057224
Cc1ccc(S(=O)(=O)N2CCC(C(=O)OCC(=O)N3CCc4ccccc43)CC2)cc1,0.896021,0.098880,-0.077974,0.172329,-0.502672,-0.195353,-0.003666,0.359869,-0.257264,0.381318,...,-0.044884,-0.074440,-0.054088,-0.007127,0.100678,-0.059102,0.229164,-0.022608,0.016921,-0.142763
COc1ccc(N2C(=O)CC(Nc3ccc(F)cc3)C2=O)c([N+](=O)[O-])c1,0.723859,-0.205432,0.217959,0.393966,0.107823,-0.036598,-0.278369,-0.081905,-0.118451,0.169892,...,0.188374,-0.154732,0.443526,-0.363257,0.111364,0.327686,0.046457,-0.164639,0.050844,0.047663
Cc1cc(C)cc(OCC(=O)O[NH+]=C(N)Cc2ccc([N+](=O)[O-])cc2)c1,0.497740,-0.259036,-0.184412,0.317811,0.105905,-0.046879,-0.234901,-0.127004,-0.102103,-0.349757,...,-0.013156,-0.082572,0.220970,-0.157787,0.059178,-0.087978,0.213287,0.089154,0.025531,-0.193025
NNC(=O)C=C1CCN(c2ncc(C(F)(F)F)cc2Cl)CC1,0.248380,-0.654509,0.707747,0.070735,-0.080659,0.100295,-0.276110,0.475557,0.303163,0.201304,...,-0.045375,-0.075370,0.023025,0.015128,0.011770,-0.092893,0.013800,0.126137,0.010645,0.052837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCNC(=O)c1nc[nH]c1C(=O)NCC,0.020729,-0.783537,0.199520,-0.339289,0.374592,-0.030632,-0.504900,-0.430618,0.095708,-0.224643,...,-0.160199,-0.013405,-0.046383,0.062624,0.046639,0.136193,-0.009248,-0.050342,-0.095663,-0.049148
CCC(C)C(NC(=O)C(C)NC(=O)CNC(=O)CNC(=O)OCc1ccccc1)C(=O)OC,1.407250,-0.226398,0.105831,0.073967,0.403355,0.316681,-0.386715,-0.401271,-0.424372,-0.604003,...,0.310731,-0.089131,0.113009,-0.046511,0.302468,0.131955,-0.159522,0.081270,0.143940,0.042307
O=C(CSc1ncnc2ccccc12)Nc1cccc(S(=O)(=O)N2CCOCC2)c1,0.538169,0.451513,0.295943,-0.040919,-0.171358,0.162549,0.074533,0.632897,-0.289320,0.210618,...,0.041557,0.020008,0.295602,-0.026589,0.206491,0.128611,-0.095513,0.108635,-0.004203,-0.087107
Cc1cc(OCCOCC[NH+](C)CCO)cc(C)c1Cl,0.158742,-0.791119,-0.405481,0.028251,0.631126,0.025393,0.508849,0.425807,0.203993,-0.427586,...,0.052538,-0.166707,0.215033,-0.212642,0.310644,0.115416,0.232212,0.008466,0.127079,-0.137654


<h2>Export the final df_reduced as csv for further use</h2>

In [30]:
df_reduced.to_csv("reduced_scaled_list_of_untested_molecs.csv")